In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df_menu = pd.read_excel('./menu.xlsx',
                        header=0,
                        dtype=str).fillna('')

list_url = df_menu['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import random
import requests
import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        part = url.split('?')[0][-12:]
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(11):
            try:
                list_appid = ['tfy6d12dc-19a5-4093-9f00-f276d950f77',
                              'picheng11-5163-4258-be21-a5629527a07',
                              'Shanghai-b2aa-4eed-82c3-b1e42e2e9179',
                              'danjidin-f8b1-4e84-adce-026944025ceb',
                              'ibay365a-b93e-46e7-8b46-6a33b716ab2a',
                              '-IBAY365-PRD-08e35c535-85b731b3']

                appid_index = random.randint(0, 5)

                developer = 'http://106.75.138.2/index.php/api/GetAppToken?auth=757F054AA22E1711B598A89EB276EDC193933EC8&appid=' + list_appid[appid_index]
                resp = requests.get(developer).text
                token = json.loads(resp)['getapptokenResponse']['token']
                
                # = = = = = = = = = = = = = = =
                
                headers = {'X-EBAY-API-IAF-TOKEN': token}
                resp = requests.get('https://open.api.ebay.com/shopping?callname=GetSingleItem&appid=' + list_appid[appid_index] + '&responseencoding=JSON&siteid=0&version=967&IncludeSelector=Details,Compatibility,ItemSpecifics&ItemID=' + part, headers=headers, proxies=Proxy.get_Proxy_Requests()).text

                info = json.loads(resp)
                if info['Ack'] != 'Success':
                    continue
                
                # = = = = = = = = = = = = = = =
                
                title = info['Item']['Title']
                
                # = = = = = = = = = = = = = = =
                
                price = info['Item']['CurrentPrice']['CurrencyID'] + ' ' + str(info['Item']['CurrentPrice']['Value'])

                # = = = = = = = = = = = = = = =
                
                sold = info['Item']['QuantitySold']
                
                # = = = = = = = = = = = = = = =
                
                if 'ItemCompatibilityList' in info['Item']:
                    vehicle = info['Item']['ItemCompatibilityList']['Compatibility']
                else:
                    vehicle = ''
                    
                # = = = = = = = = = = = = = = =
                
                dict_vehicle = {}
                for list_name_value in vehicle:
                    make = ''
                    model = ''
                    year = ''
                    for name_value in list_name_value['NameValueList']:
                        if 'Name' in name_value and 'Make' in name_value['Name']:
                            make = name_value['Value'][0]
                        elif 'Name' in name_value and 'Model' in name_value['Name']:
                            model = name_value['Value'][0]
                        elif 'Name' in name_value and 'Year' in name_value['Name']:
                            year = int(name_value['Value'][0])

                    if make not in dict_vehicle:
                        dict_vehicle[make] = {model: [year]}
                    else:
                        if model not in dict_vehicle[make]:
                            dict_vehicle[make][model] = [year]
                        else:
                            dict_vehicle[make][model].append(year)

                list_vehicle = []
                for make in dict_vehicle:
                    for model in dict_vehicle[make]:
                        list_vehicle.append(make + ' ' + model + ' ' + str(min(dict_vehicle[make][model])) + '-' + str(max(dict_vehicle[make][model])))

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                if 'GalleryURL' in info['Item']:
                    src = info['Item']['GalleryURL']
                elif 'PictureURL' in info['Item']:
                    if len(info['Item']['PictureURL']) == 0:
                        src = ''
                    else:
                        src = info['Item']['PictureURL'][0]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         '序号': '',
                                         'Item_Number': part,
                                         'Title': title,
                                         'Price': price,
                                         'Sold': sold,
                                         'Vehicle': vehicle,
                                         'Pic': '',
                                         'Url': url,
                                         'Src': src}])
                
                status = 'ok'
                
                # = = = = = = = = = = = = = = =
                
                if 'ItemSpecifics' in info['Item']:
                    details = info['Item']['ItemSpecifics']['NameValueList']
                else:
                    details = ''
                
                # = = = = = = = = = = = = = = =
                
                for name_value in details:
                    df_temp[name_value['Name'].lower()] = ';'.join(name_value['Value'])
                    
                break
                
            except:
                time.sleep(3)
                continue
                
        # = = = = = = = = = = = = = = =

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Item_Number': part,
                                     'Url': url}])

        # = = = = = = = = = = = = = = =

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
df.to_excel('./part_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：22

133137610220  <->  [ok] - 剩余数量：2
133380881458  <->  [ok] - 剩余数量：1
133763993990  <->  [ok] - 剩余数量：0
132617012107  <->  [ok] - 剩余数量：0
132761565549  <->  [ok] - 剩余数量：0
133491866811  <->  [ok] - 剩余数量：0
133037336945  <->  [ok] - 剩余数量：0
134743579896  <->  [ok] - 剩余数量：0
133787083591  <->  [ok] - 剩余数量：0
133102294289  <->  [ok] - 剩余数量：0
133463914728  <->  [ok] - 剩余数量：0
134202507501  <->  [ok] - 剩余数量：0
132851619983  <->  [ok] - 剩余数量：0
133764955937  <->  [ok] - 剩余数量：0
133225841644  <->  [ok] - 剩余数量：0
133721506770  <->  [ok] - 剩余数量：0
133895024695  <->  [ok] - 剩余数量：0
133515394591  <->  [ok] - 剩余数量：0
133516892439  <->  [ok] - 剩余数量：0
133084374156  <->  [ok] - 剩余数量：0
133599883987  <->  [ok] - 剩余数量：0
134085019439  <->  [ok] - 剩余数量：0

爬虫输出中

爬虫结束


D:\venv\Lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:
